In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
qtd_msg = 1

lista = list(range(qtd_msg))
print(lista)

[0]


In [3]:
qtd_msg = 1

lista = range(qtd_msg)
print(lista)

range(0, 1)


In [4]:
qtd_msg = 4
lista = list(range(qtd_msg - 1, -1, -1))
for p, i in enumerate(lista):
    print("posição", i)
    print("indice", p+1)
    

posição 3
indice 1
posição 2
indice 2
posição 1
indice 3
posição 0
indice 4


In [5]:
base_clientes = pd.read_csv('../data/base_clientes.csv')

In [6]:
base_clientes.head()

,cpf,nome,segmento,qtd_cont,vlr_total_div,max_dias_atraso
0,42006925890,Bryan Campos,Van Gogh,10,5000,60
1,34366343878,Juan Moura,Private,5,2000,30
2,72388905850,Ana Sophia da Cruz,Van Gogh,1,2293,28
3,85044167398,Marina Moraes,Especial,7,2685,42
4,86406862031,Marcos Vinicius Cardoso,Van Gogh,13,9173,50


In [7]:
# Especificar o caminho para o diretório onde o banco de dados está localizado
path = "../data/"

# Especificar o valor do CPF como uma string entre aspas simples
cpf_value = 42006925890
query = f"SELECT * FROM customer_database_table WHERE cpf = '{cpf_value}'"

# Usar a declaração `with` para gerenciar a conexão ao banco de dados SQLite
with sqlite3.connect(os.path.join(path, 'customer_database.sqlite')) as conn:
    # Executar a consulta SQL e ler o resultado em um DataFrame do pandas
    base = pd.read_sql_query(query, conn)

In [8]:
base.head()

,cpf,nome,segmento,qtd_cont,vlr_total_div,max_dias_atraso
0,42006925890,Bryan Campos,Van Gogh,10,5000,60


In [9]:
base["nome"].values

array(['Bryan Campos'], dtype=object)

In [7]:
# Teste Mongo

from pymongo import MongoClient

# Conectar ao servidor MongoDB
client = MongoClient('localhost', 27017)

# Acessar o banco de dados de teste
db = client.test

# Inserir um documento de teste
result = db.test_collection.insert_one({'name': 'teste', 'value': 42})

print(f"Documento inserido com ID: {result.inserted_id}")

Documento inserido com ID: 668540b0c7fc8992e54be205


In [11]:
def conectar_mongo():
    # Verifica se a conexão já foi criada
    if not hasattr(conectar_mongo, "collection"):
        try:
            # Se não, cria uma nova conexão
            conectar_mongo.client = MongoClient('mongodb://localhost:27017/', serverSelectionTimeoutMS=5000)
            conectar_mongo.client.server_info()  # Tenta obter informações do servidor para testar a conexão
            conectar_mongo.db = conectar_mongo.client['db_conversas']
            conectar_mongo.collection = conectar_mongo.db['clientes_conversas']
        except Exception as e:
            print(f"Erro ao conectar ao MongoDB: {e}")
            conectar_mongo.client = None
            return None
    else:
        print("Conexão existente com o MongoDB")

    return conectar_mongo.client, conectar_mongo.db, conectar_mongo.collection

In [14]:
# conectar_mongo.collection = None
del conectar_mongo.collection

In [15]:
conectar_mongo()

(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000),
 Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000), 'db_conversas'),
 Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000), 'db_conversas'), 'clientes_conversas'))

## Testando funções MongoDB

In [20]:
from pymongo import MongoClient

# Visualizaçao do json
import json
from bson import ObjectId

In [21]:
def conectar_mongo():
    try:
        client = MongoClient('mongodb://localhost:27017/', serverSelectionTimeoutMS=5000)
        client.server_info()  # Gera uma exceção se não conseguir conectar
        print("Conexão com MongoDB bem-sucedida.")
        if client:
            db = client['meu_banco_de_dados']
            collection = db['clientes_conversas']
        return client, db, collection
    except Exception as e:
        print(f"Erro ao conectar ao MongoDB: {e}")
        return None

In [22]:
client, db, collection = conectar_mongo() # Conectar ao banco de dados

Conexão com MongoDB bem-sucedida.


In [23]:
# Função personalizada para serializar ObjectId
class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, ObjectId):
            return str(obj)
        return json.JSONEncoder.default(self, obj)

In [24]:
def formatar_json(documento):
    return json.dumps(documento, indent=4, ensure_ascii=False, cls=JSONEncoder)

In [25]:
# Consultar todos os documentos na coleção
def consultar_todos():
    documentos = collection.find()
    for doc in documentos:
        print(formatar_json(doc))

# Consultar documentos com uma condição
def consultar_condicao(cpf):
    documento = collection.find_one({"cpf": cpf})
    if documento:
        formatar_json(documento)
    else:
        print(f"Nenhum documento encontrado para o CPF {cpf}")

In [26]:
def inserir_cliente(cpf, assunto, dt_hr_ini):
    cliente = collection.find_one({"cpf": cpf})
    # Inserir novo cliente se não existir
    if not cliente:
        collection.insert_one({
            "cpf": cpf,
            "conversas": [
                {
                    "assunto": assunto,
                    "chats": [
                        {
                            "data_hora_inicio": dt_hr_ini,
                            "mensagens": []
                        }
                    ]
                }
            ]
        })
    else:
        # Inserir nova conversa caso o cliente já existir na base
        assunto_existe = collection.find_one({"cpf": cpf, "conversas.assunto": assunto})
        if not assunto_existe:
            print("assunto novo")
            # Adicionar um novo assunto com o chat inicial
            collection.update_one(
                {"cpf": cpf},
                {"$push": {
                    "conversas": {
                        "assunto": assunto,
                        "chats": [
                            {
                                "data_hora_inicio": dt_hr_ini,
                                "mensagens": []
                            }
                        ]
                    }
                }}
            )
        else:
            # Adicionar um novo chat ao assunto existente
            collection.update_one(
                {"cpf": cpf, "conversas.assunto": assunto},
                {"$push": {
                    "conversas.$.chats": {
                        "data_hora_inicio": dt_hr_ini,
                        "mensagens": []
                    }
                }}
            )


In [27]:
cpf = 42006925890
assunto = "Reclamação"
dt_hr_ini = "2024-06-27"

In [28]:
# dados_cliente = {
#     "nome": "João da Silva",
#     "email": "caique@gmail.com",
#     "telefone": "11999999999"
# }

In [29]:
# dados_oferta = {
#     "data_ref": "2024-06-27",
#     "produto": "Smartphone",
#     "preco": 1500.00,
#     "desconto": 100.00
# }

In [33]:
inserir_cliente(cpf, assunto, dt_hr_ini)

In [34]:
consultar_todos()
# consultar_condicao(42006925890)

{
    "_id": "66857eeab339da29e9438ead",
    "cpf": 42006925890,
    "conversas": [
        {
            "assunto": "Reclamação",
            "chats": [
                {
                    "data_hora_inicio": "2024-06-27",
                    "mensagens": []
                }
            ]
        }
    ]
}


In [31]:
collection.delete_many({})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [35]:
def inserir_mensagem(cpf, assunto, dt_hr_ini, dict_mensagem):
    client, db, collection = conectar_mongo()  # Conectar ao banco de dados
    collection.update_one(
        {"cpf": cpf},
        {"$push": {"conversas.$[conversa].chats.$[chat].mensagens": dict_mensagem}},
        array_filters=[
            {"conversa.assunto": assunto},
            {"chat.data_hora_inicio": dt_hr_ini}
        ]
    )

In [36]:
dict_mensagem = {
    "id": 1,
    "data_hora": "2024-06-27 10:30:00",
    "mensagem_cliente": "Iphone 24",
    "sugestao_ia": "Esse produto não existe",
    "rating_sugestao_ia": 4,
    "resposta_final_operador": "Sr. João, o produto correto é o Iphone 15",
}

In [37]:
inserir_mensagem(cpf, assunto, dt_hr_ini, dict_mensagem)

Conexão com MongoDB bem-sucedida.


In [38]:
consultar_todos()

{
    "_id": "66857eeab339da29e9438ead",
    "cpf": 42006925890,
    "conversas": [
        {
            "assunto": "Reclamação",
            "chats": [
                {
                    "data_hora_inicio": "2024-06-27",
                    "mensagens": [
                        {
                            "id": 1,
                            "data_hora": "2024-06-27 10:30:00",
                            "mensagem_cliente": "Iphone 24",
                            "sugestao_ia": "Esse produto não existe",
                            "rating_sugestao_ia": 4,
                            "resposta_final_operador": "Sr. João, o produto correto é o Iphone 15"
                        }
                    ]
                }
            ]
        }
    ]
}


In [41]:
documentos = collection.find()

In [42]:
documentos

In [43]:
# Percorre os documentos para extrair as informações necessárias
documentos = collection.find()
conversas_lista = []
for documento in documentos:
    cpf = documento["cpf"]
    for conversa in documento["conversas"]:
        assunto = conversa["assunto"]
        for chat in conversa["chats"]:
            data_hora_inicio = chat["data_hora_inicio"]
            conversas_lista.append(f"{data_hora_inicio} - {cpf} - {assunto}")

In [44]:
# Ordena a lista por data
conversas_lista.sort()

# Exibe a lista ordenada
for conversa in conversas_lista:
    print(conversa)

2024-06-27 - 34343434343 - Reclamação
2024-06-27 - 34343434343 - Reclamação
2024-06-27 - 42006925890 - Reclamação


In [ ]:
# Percorre o dicionário para extrair as informações necessárias
for pessoa in dicionario:
    cpf = pessoa["cpf"]
    for conversa in pessoa["conversas"]:
        assunto = conversa["assunto"]
        for chat in conversa["chats"]:
            data_hora_inicio = chat["data_hora_inicio"]
            conversas_lista.append(f"{data_hora_inicio} - {cpf} - {assunto}")

# Ordena a lista por data
conversas_lista.sort()

# Exibe a lista ordenada
for conversa in conversas_lista:
    print(conversa)

In [2]:
lista = ["2024-06-27 - 42006925890 - Reclamação", "2024-06-27 - 42006925890 - Boleto"]

In [4]:
lista[:]

['2024-06-27 - 42006925890 - Reclamação', '2024-06-27 - 42006925890 - Boleto']

In [49]:
lista2 = []
for i in lista:
    lista2.append(i.split(" - "))

In [1]:
lista2

NameError: name 'lista2' is not defined

Criando conversas fictícias para testar a sidebar

In [6]:
posicoes_lista = []